# 🔊 Proyecto 2  _(detección de emociones por voz)_ 🔊 

💡Integrantes💡

✨Laura Sofía Ortiz Arcos
✨David Santiago Flórez Alsina

In [1]:
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt
import random 

from sklearn.model_selection import train_test_split
from sklearn                 import preprocessing
from torch.utils.data        import DataLoader
from torch.utils.data        import TensorDataset
from sklearn.preprocessing   import StandardScaler, MinMaxScaler

import torch
import sklearn
import torch.nn            as nn
import torch.nn.functional as F
import torch.utils.data    

### Cargar y preparar los datos

Para esta parte primero cargaremos los datos sobre señales de voz, de acuerdo al sentimiento de dicha voz.

In [2]:
df = pd.read_csv('emotions_by_voice_registers.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,X,meanfreq,sd,median,Q25,Q75,IQR,skew,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0,1,1,0.181338,0.060495,0.187476,0.126197,0.233586,0.107389,0.869088,...,0.181338,0.137742,0.023022,0.271186,0.777344,0.085938,6.226562,6.140625,0.116586,sad
1,1,2,2,0.186897,0.062260,0.195070,0.130847,0.243987,0.113140,1.191767,...,0.186897,0.121811,0.018412,0.271186,0.930339,0.085938,4.000000,3.914062,0.144983,sad
2,2,3,3,0.189102,0.062901,0.204945,0.131422,0.249978,0.118556,1.312690,...,0.189102,0.123758,0.083333,0.262295,0.332386,0.085938,0.625000,0.539062,0.334783,sad
3,4,5,5,0.183036,0.060051,0.174115,0.129949,0.236967,0.107017,1.096409,...,0.183036,0.128469,0.044693,0.258065,1.012019,0.085938,5.468750,5.382812,0.304910,sad
4,5,6,6,0.168793,0.057910,0.156266,0.116783,0.216326,0.099543,1.386837,...,0.168793,0.109720,0.022472,0.235294,0.228795,0.093750,0.750000,0.656250,0.306777,sad


_**Nos damos algo de información sobre de qué trata nuestro dataset y cómo es**_

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909 entries, 0 to 908
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    909 non-null    int64  
 1   Unnamed: 0.1  909 non-null    int64  
 2   X             909 non-null    int64  
 3   meanfreq      909 non-null    float64
 4   sd            909 non-null    float64
 5   median        909 non-null    float64
 6   Q25           909 non-null    float64
 7   Q75           909 non-null    float64
 8   IQR           909 non-null    float64
 9   skew          909 non-null    float64
 10  kurt          909 non-null    float64
 11  sp.ent        909 non-null    float64
 12  sfm           909 non-null    float64
 13  mode          909 non-null    float64
 14  centroid      909 non-null    float64
 15  meanfun       909 non-null    float64
 16  minfun        909 non-null    float64
 17  maxfun        909 non-null    float64
 18  meandom       909 non-null    

_**Luego eliminamos los atributos tipo índice, en este caso son:**_ 'Unnamed 0', 'Unnamed 0.1' y 'X'.

In [4]:
X = df.drop(labels = ['Unnamed: 0', 'Unnamed: 0.1', 'X'], axis = 1)
X

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.181338,0.060495,0.187476,0.126197,0.233586,0.107389,0.869088,2.863717,0.923566,0.307220,...,0.181338,0.137742,0.023022,0.271186,0.777344,0.085938,6.226562,6.140625,0.116586,sad
1,0.186897,0.062260,0.195070,0.130847,0.243987,0.113140,1.191767,3.878650,0.918848,0.298859,...,0.186897,0.121811,0.018412,0.271186,0.930339,0.085938,4.000000,3.914062,0.144983,sad
2,0.189102,0.062901,0.204945,0.131422,0.249978,0.118556,1.312690,4.589995,0.919519,0.313069,...,0.189102,0.123758,0.083333,0.262295,0.332386,0.085938,0.625000,0.539062,0.334783,sad
3,0.183036,0.060051,0.174115,0.129949,0.236967,0.107017,1.096409,3.680995,0.921361,0.329295,...,0.183036,0.128469,0.044693,0.258065,1.012019,0.085938,5.468750,5.382812,0.304910,sad
4,0.168793,0.057910,0.156266,0.116783,0.216326,0.099543,1.386837,5.031744,0.926238,0.337047,...,0.168793,0.109720,0.022472,0.235294,0.228795,0.093750,0.750000,0.656250,0.306777,sad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,0.244013,0.035477,0.254385,0.229653,0.265573,0.035920,2.214752,7.565052,0.821874,0.136933,...,0.244013,0.202433,0.028829,0.271186,0.616536,0.210938,1.609375,1.398438,0.281869,happy
905,0.235383,0.045303,0.248974,0.220745,0.264233,0.043488,2.474743,9.959019,0.848109,0.236957,...,0.235383,0.189293,0.031250,0.275862,1.115723,0.265625,5.500000,5.234375,0.167861,happy
906,0.231211,0.044793,0.234847,0.221477,0.262090,0.040613,2.607668,10.698821,0.848702,0.241998,...,0.231211,0.171805,0.022346,0.275862,1.070801,0.265625,4.554688,4.289062,0.214936,happy
907,0.213587,0.082267,0.249435,0.207680,0.268538,0.060858,3.460579,18.034614,0.882544,0.425394,...,0.213587,0.155277,0.020592,0.275862,1.724888,0.273438,6.812500,6.539062,0.238857,happy


Nos hacemos unas _**funciones para estandarizar**_ y probar el rendimiento del modelo con las distintas estandarizaciones, y sin alguna estandarización.

In [5]:
def data_scaling(data: pd.DataFrame,
                 scaler: sklearn.preprocessing._data,
                 scaling_columns     = [], 
                 not_scaling_columns = []):
    
    """
    Scales the specified columns in the input DataFrame using the given scaler object.
    
    INPUT:
        - data:                Pandas DataFrame to be scaled
        - scaler:              Scaler object from sklearn.preprocessing
        - scaling_columns:     List of column names to scale
        - not_scaling_columns: List of column names to not scale
        
    OUTPUT: 
        - pd.DataFrame:        A scaled copy of the input data with the specified columns scaled
    """

    # if the user provided all the columns he wishes to scale 
    if scaling_columns:
        
        data[scaling_columns].iloc[:] = scaler.fit_transform(data[scaling_columns].iloc[:].to_numpy())

    # if user provided only the columns he doesn't want to scale
    elif not_scaling_columns:
        
        used_cols = data.columns.difference(not_scaling_columns)
        data.loc[:, used_cols] = scaler.fit_transform(data.loc[:, used_cols].to_numpy())

    return data

In [6]:
X  = data_scaling(data = X, scaler = StandardScaler(), not_scaling_columns = ['label'])

También _**creamos variables dummy**_ para la variable categórica.

In [7]:
X = pd.get_dummies(X, columns = ['label'])
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909 entries, 0 to 908
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   meanfreq     909 non-null    float64
 1   sd           909 non-null    float64
 2   median       909 non-null    float64
 3   Q25          909 non-null    float64
 4   Q75          909 non-null    float64
 5   IQR          909 non-null    float64
 6   skew         909 non-null    float64
 7   kurt         909 non-null    float64
 8   sp.ent       909 non-null    float64
 9   sfm          909 non-null    float64
 10  mode         909 non-null    float64
 11  centroid     909 non-null    float64
 12  meanfun      909 non-null    float64
 13  minfun       909 non-null    float64
 14  maxfun       909 non-null    float64
 15  meandom      909 non-null    float64
 16  mindom       909 non-null    float64
 17  maxdom       909 non-null    float64
 18  dfrange      909 non-null    float64
 19  modindx 

### Creación de los conjuntos de entrenamiento, validación y test

Ahora, creamos los conjuntos de entrenamiento, validación y test. Vamos a usar una proporción de:

- Para test un 10%.
- Para validación un 10%.
- Y el resto para train.

También creamos una _**clase Dataset**_ que nos prepara los datos para alimentar la red neuronal, convirtiendolos a parejas ordenadas de tensores conteniento los atributos y la variable objetivo. 

In [8]:
class Dataset():
 
  def __init__(self,
               df: pd.DataFrame,
               target_column: list):

    y = df[target_column].values
    X = df.drop(target_column,axis=1).values

    self.X = torch.tensor(X, dtype=torch.float32)
    self.y = torch.tensor(y, dtype=torch.float32)
 
  def __len__(self):
    return len(self.y)
   
  def __getitem__(self,idx):
    return self.X[idx], self.y[idx]

In [9]:
def train_test_val_split(data: pd.DataFrame, 
                         proportions = [0.8, 0.1, 0.1]):

    """
        Splits the input DataFrame into train, test, and validation sets based on the given proportions.

        INPUT:
            - data:        The dataframe to be splitted

            - proportions: List of proportions of size of each partition of the dataframe, first element 
                           of the list corresponds to the proportion of train, the 2nd one indicates the
                           test proportion, and the last element is the validation proportion

        OUTPUT: 
            - (pd.DataFrame, pd.DataFrame, pd.DataFrame): in the form
              (train_dataset, test_dataset, validation_dataset)
    """

    data_len = len(data)

    #end of train
    train_indx = int(proportions[0]*data_len)

    #end of test
    test_indx = train_indx + int(proportions[1]*data_len)

    out = (data.iloc[:train_indx],
           data.iloc[train_indx+1:test_indx],
           data.iloc[test_indx+1:-1])
           
    return out


In [10]:
train, test, val = train_test_val_split(X)

### Preparación de los datos para alimentar la red

Continuamos preparando los datos, convirtiendolos a tensores, y creando los dataloaders.

In [11]:
outcome_vars = ['label_angry', 'label_happy', 'label_sad']

# Use the Dataset class to prepare each array in the form of tensors
train_sec = Dataset(train, outcome_vars)
test_sec  = Dataset(test, outcome_vars)
val_sec   = Dataset(val, outcome_vars)

Acontinuación puede ver el tamaño de nuestro set de entrenamiento, testeo y validación:

In [12]:
s = f"train: {train_sec.X.shape}, test: {test_sec.X.shape}, val: {val_sec.X.shape}"
print(s)

train: torch.Size([727, 20]), test: torch.Size([89, 20]), val: torch.Size([90, 20])


In [13]:
# Define the DataLoaders to load the information in batches
train_data = DataLoader(
    train_sec,
    batch_size = 16,
    shuffle    = False,
 )

test_data = DataLoader(
    test_sec,
    batch_size  = 16,
    shuffle     = False,
    #num_workers = 0,
    #collate_fn  = None,
    #pin_memory  = False,
 )

val_data = DataLoader(
    val_sec,
    batch_size  = 16,
    shuffle     = False,
    #num_workers = 0,
    #collate_fn  = None,
    #pin_memory  = False,
 )

### Definición de la clase Net, el optimizador, la función de costo y el modelo

Para hacer nuestra red neuronal decidimos darle la posibilidad de generar distintas estructuras para ella, nuestro método para hacer esto automatizadamente consiste guiarnos de una función (seleccionada por nosotros), para definir la cantidad de neuronas que queremos tener en el número de capas que hayamos decidido poner.

Si por ejemplo queremos una arquitectura que empieza con su primera capa con 20 neuronas, aumenta a 40 y sigue en 40 y después baja a 20 y llega hasta 3 podríamos guiarnos de esta función e ir mapeando ciertos puntos de ella:


<iframe src="https://www.desmos.com/calculator/dy1goxwgu2?embed" width="800" height="400" style="border: 1px solid #ccc" frameborder=50></iframe> (hay un link con la gráfica, pero no sabemos porque Jupyter no la muestra. Igual en el .html se puede ver la gráfica por si acaso)


En este caso estamos usando la función 

$$f\left(x\right)\ =\ 10\cdot\sqrt{x}\ -\ x\ +\ 20\ \left\{0\le x\le130\right\}$$

La cuál fue definida a nuestro antojo para cumplir lo que queríamos con nuestra red neuronal, que básicamente es que el número de neuronas por capa aumente un poquito y luego caiga hasta llegar al número de neuronas de salida. Con esta función definida lo que resta es evaluar el conjunto 

$$\vec{x} = \{ i*\Delta X,\quad i \in \mathbb{Z}^{*}, \quad \Delta X = \frac{\text{inicio dominio} - \text{fin  dominio}}{\text{num. capas}} \}$$



In [14]:
def g1(domain: list)-> np.array:
    """
    start at 20, grow and then decay function

    INPUT:
        - domain:   List of numeric values to be processed
    OUTPUT: 
        - np.array: Array of integers resulting from the application of the mathematical formula
    """

    return ((10*np.sqrt(domain)) - domain + 20).astype(np.int32)


def g2(domain: list) -> np.array:
    """
    f(x) = 20 -x, kind of function
    
    INPUT:
        - domain:   list of numeric values to be processed
    OUTPUT: 
        - np.array: array of integers resulting from the application of the mathematical formula
    """
    
    # apply the mathematical formula to each item in the list and convert the resulting array to integers
    return (20 - (0.05 * np.array(domain) ** 3)).astype(np.int32)


def g3(domain: list)-> np.array:
    """
    Constant function.
    
    INPUT:
        - domain:   list of numeric values (not used in this function)
    OUTPUT: 
        - np.array: array of integers where each element is 20
    """

    return np.array([20 for i in range(len(domain))])

Con nuestra abstracción anterior, podemos proseguir a crear diferentes modelos simplemente cambiando la función del comportamiento del numero de neuronas. Así definimos una función para construir diferentes modelos:

In [15]:
def build_different_models(input_sz: int,
                           output_sz: int,
                           max_layers: int, 
                           envelope_f: callable, 
                           lim: int) -> list:

    """
    Builds a list of different neural network models with varying numbers of hidden layers and neurons.
    
    INPUT:
        - input_sz:     integer specifying the size of the input layer.
        - output_sz:    integer specifying the size of the output layer.
        - max_layers:   integer specifying the maximum number of hidden layers allowed.
        - envelope_f:   callable that takes in an array of sample points and returns an array of integers
                        specifying the number of neurons in each hidden layer of a neural network model.
        - lim:          integer specifying the maximum value of the sample points array, this is the 
                        limit of the domain of the function.
    
    OUTPUT: 
        - list of nn.ModuleList: list of different neural network models with varying architectures
    """


    # list of lists, each list contains the number of neurons
    # the model will have on each layer
    hidden_layers_structure = []


    # here we build the different neural architectures
    for i in range(2, max_layers+1):
        #delta = int((input_sz - output_sz)/i)
        sample_points = np.linspace(0, lim, i)
        structure     = envelope_f(sample_points) 
        hidden_layers_structure.append(structure)

    # list of models
    models = []

    # populate the list of models
    for structure in hidden_layers_structure:

        # Define your layers for the model 
        model = nn.ModuleList()

        # first layer
        model.append(nn.Linear(input_sz, structure[0]))

        # hidden layers
        for i, n in enumerate(structure[:-1]):
            model.append(nn.Linear(structure[i], structure[i+1]))

        
        # output layer
        model.append(nn.Linear(structure[-1], output_sz))
        model.append(nn.Linear(3, output_sz))

        models.append(model)
        
    return models

Ahora continuamos definiendo nuestra _**clase NeuralNetwork**_ con la estructura de la red neuronal (haciendo uso de capas lineales y las diferentes funciones de activación).

In [16]:
class NeuralNetwork(nn.Module):
    """
        A class that defines a neural network with a configurable number of layers
        and activation function.

        Attributes:
            - flatten (nn.Flatten):   Flattens the input to a 1D tensor.
            - model (nn.ModuleList):  Contains the layers of the neural network.
            - output (nn.Softmax):    Applies softmax function to the output of the last layer.
            - activation (nn.Module): Activation function used in the hidden layers.

        Methods:
            - __init__():             Initializes the NeuralNetwork class.
            - forward(x):             Forward pass through the neural network.
            - set_model(mod):         Sets the model layers.
            - set_activation(act):    Sets the activation function used in the hidden layers.
            - make_prediction(x):     Generates predictions for a given input.

        Example:
            >>> model = NeuralNetwork()
            >>> model.set_activation(nn.LeakyReLU())
            >>> model.set_model([nn.Linear(10, 20), nn.Linear(20, 2)])
            >>> predictions = model.make_prediction(torch.randn(5, 10).numpy())
    """

    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.model   = nn.ModuleList()

        self.output     = nn.Softmax()
        self.activation = nn.ReLU()

    def forward(self, x):
        """
        Forward pass through the neural network.

        Args:
            - x (torch.Tensor):      Input tensor.

        Returns:
            - output (torch.Tensor): Output tensor of the neural network.
        """
        x = self.flatten(x)

        for layer in self.model:
            x = self.activation(layer(x))

        return self.output(x)
    
    def set_model(self, mod):
        """
        Sets the model layers.

        Args:
            - mod (list): List containing the neural network layers.
        """
        self.model = mod

    def set_activation(self, act: torch.nn.modules.activation):
        """
        Sets the activation function used in the hidden layers.

        Args:
            - act (nn.Module): Activation function.
        """
        self.activation = act

    def make_prediction(self, x: np.array) -> np.array:
        """
        Generates predictions for a given input.

        Args:
            - x (np.array):     Input tensor.

        Returns:
            - preds (np.array): Predictions tensor of the neural network.
        """
        preds = self.forward(x).argmax(dim = 1)
        return preds

Imprimimos nuestros posibles candidatos a modelo.

In [17]:
candidate_models = build_different_models(input_sz   = 20,
                                          output_sz  = 3,
                                          max_layers = 5, 
                                          envelope_f = g1, 
                                          lim = 130)
candidate_models 

[ModuleList(
   (0): Linear(in_features=20, out_features=20, bias=True)
   (1): Linear(in_features=20, out_features=4, bias=True)
   (2): Linear(in_features=4, out_features=3, bias=True)
   (3): Linear(in_features=3, out_features=3, bias=True)
 ),
 ModuleList(
   (0): Linear(in_features=20, out_features=20, bias=True)
   (1): Linear(in_features=20, out_features=35, bias=True)
   (2): Linear(in_features=35, out_features=4, bias=True)
   (3): Linear(in_features=4, out_features=3, bias=True)
   (4): Linear(in_features=3, out_features=3, bias=True)
 ),
 ModuleList(
   (0): Linear(in_features=20, out_features=20, bias=True)
   (1): Linear(in_features=20, out_features=42, bias=True)
   (2): Linear(in_features=42, out_features=26, bias=True)
   (3): Linear(in_features=26, out_features=4, bias=True)
   (4): Linear(in_features=4, out_features=3, bias=True)
   (5): Linear(in_features=3, out_features=3, bias=True)
 ),
 ModuleList(
   (0): Linear(in_features=20, out_features=20, bias=True)
   (1):

Revisamos que estemos usando GPU y definimos el dispositivo

In [18]:
gpu_avail = torch.cuda.is_available()
print(f"Is the GPU available? {gpu_avail}")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)

Is the GPU available? True
Device cuda


Luego definimos el modelo, el optimizador y la función de costo

In [19]:
model = NeuralNetwork()
model.set_model(candidate_models[0])

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
optimizer.zero_grad()

criterion = nn.CrossEntropyLoss()

### Entrenando la red y analizando el mejo modelo

Luego de haber preparado los datos, definir la arquitectura, y de así entrenar la red, vamos a definir la _**función de entrenamiento**_ para guardar nuestro mejor modelo. 

In [24]:
# pass the model to the GPU device
model.to(device)

def train_model(model,
                optimizer,
                loss_module,
                train_loader,
                valid_loader, 
                num_epochs):
    """
    Trains the specified model using the given data and hyperparameters, and saves the model with 
    the smallest validation error.

    INPUT:
        - model:        the PyTorch model to be trained
        - optimizer:    the PyTorch optimizer used to update the model's parameters
        - loss_module:  the PyTorch loss function used to calculate the training loss
        - train_loader: the PyTorch DataLoader containing the training data
        - valid_loader: the PyTorch DataLoader containing the validation data
        - num_epochs:   the number of epochs to train the model for

    OUTPUT:
        None
    """
  
    # Let's find the smallest validation error value.
    # So initialize it to 'infinity'
    valid_loss_min = np.inf  
  
    for i in range(num_epochs):
      
        # We put the model in training mode.        
        # It is important in other architectures such as convolutional networks.
        model.train()  
        train_loss = 0.0
        v_loss = 0.0

        for data, target in train_loader:
            # move the attribute and label tensors to the GPU device
            inputs, labels = data.to(device), target.to(device)
            
            # reset the gradients
            optimizer.zero_grad()

            # forward pass: calculate the output for the input data.
            outputs = model(inputs)

            # calculate the batch loss
            loss = loss_module(outputs, labels)

            # backpropagation: gradient calculation
            loss.backward()

            # update parameters
            optimizer.step()

            # update cost account across batches
            train_loss += loss.item()*data.size(0)

        train_loss = train_loss/len(train_loader.dataset) 

        # We put the model in evaluation mode.
        model.eval() 

        # we are going to evaluate the trained model, calculating predictions with the validation set
        for data,target in valid_loader:
            data=data.to(device)
            target=target.to(device)
            output=model(data)
            valid_loss= loss_module(output, target)
            valid_loss += loss.item()*data.size(0)

        valid_loss = valid_loss/len(valid_loader.dataset)
    
        # print training and validation statistics
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            i, train_loss, valid_loss))
        

        # We save the model with the smallest validation error.
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), 'model_bikeshare.pt')
            valid_loss_min = valid_loss

Imprimimos los parámetros de los modelos.

In [35]:
model = NeuralNetwork()
model.set_model(candidate_models[3])
optimizer.zero_grad()

train_model(model, optimizer, criterion, train_data, val_data, 50)  

/tmp/ipykernel_5201/2872644584.py:49: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.output(x)


Epoch: 0 	Training Loss: 1.098303 	Validation Loss: 0.137598
Validation loss decreased (inf --> 0.137598).  Saving model ...
Epoch: 1 	Training Loss: 1.098303 	Validation Loss: 0.137598
Validation loss decreased (0.137598 --> 0.137598).  Saving model ...
Epoch: 2 	Training Loss: 1.098303 	Validation Loss: 0.137598
Validation loss decreased (0.137598 --> 0.137598).  Saving model ...
Epoch: 3 	Training Loss: 1.098303 	Validation Loss: 0.137598
Validation loss decreased (0.137598 --> 0.137598).  Saving model ...
Epoch: 4 	Training Loss: 1.098303 	Validation Loss: 0.137598
Validation loss decreased (0.137598 --> 0.137598).  Saving model ...
Epoch: 5 	Training Loss: 1.098303 	Validation Loss: 0.137598
Validation loss decreased (0.137598 --> 0.137598).  Saving model ...
Epoch: 6 	Training Loss: 1.098303 	Validation Loss: 0.137598
Validation loss decreased (0.137598 --> 0.137598).  Saving model ...
Epoch: 7 	Training Loss: 1.098303 	Validation Loss: 0.137598
Validation loss decreased (0.13759

Y finalmente, evaluamos el desempeño del modelo que creemos que es mejor.

In [36]:
def test_model_performance(model: NeuralNetwork, test_set: Dataset): 
    """
    Evaluates the performance of a neural network model on a given test dataset.

    INPUT:
        - model:    A neural network model to be tested.
        - test_set: A dataset containing test samples and corresponding labels.

    OUTPUT:
        - accuracy: The accuracy of the model on the test dataset, expressed as a percentage.
    """

    model.eval()

    # we feed the model with the data in order to get the outputs
    # and with all the outputs predicted by de model, we proceed
    # to get the maximum of each prediction category
    preds = model(test_set.X).argmax(dim = 1)

    # now we compare with the expected outcomes
    target = test_set.y.argmax(dim = 1)

    # we compare both and get the total number of correct answers given 
    # by the model
    correct = np.sum( np.array((preds == target).flatten()) ).astype('int32')
    accuracy = correct/len(preds) 

    return accuracy*100

In [37]:
test_model_performance(model, test_sec)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

### Mejoras para el modelo

Como se puede observar anteriormente, no tenemos un buen accuracy como estamos definiendo nuestro modelo, por lo anterior decidimos crear una función que nos ayudara a ver cuál modelo sería el mejor, para esto, la función nos permite crear diferentes arquitecturas de la red neuronal, y también utilizar varias funciones de activación.

In [ ]:
def try_everything(input_sz:   int,
                   output_sz:   int,
                   funs:       list,
                   lims:       list,
                   acts:       list,
                   max_layers: int,
                   n_epochs:   int, 
                   criterion:  torch.nn.modules.loss, 
                   data:       tuple):
    """
    INPUT:
        - input_sz:   the size of the input layer
        - output_sz:  the size of the output layer
        - funs:       a list of functions to be used to generate the hidden layer structures
        - lims:       a list of the upper limits of the range of the input to each function in funs
        - acts:       a list of activation functions to be used
        - max_layers: the maximum number of layers allowed in the neural network
        - n_epochs:   the number of epochs to train the neural network for
        - criterion:  the loss function to be used
        - data:       a tuple of three datasets, consisting of training data,
                      validation data, and test data
        
    OUTPUT:
        - a tuple of the best score achieved by any of the models tested and the model that achieved 
          that score
    """

    train_sec, val_sec, test_sec = data

    # list of models
    models = []

    # list of each model score
    scores = []  

    for act in acts:
        for fun, lim in zip(funs, lims):
            for i in range(2, max_layers+1):

                sample_points = np.linspace(0, lim, i)
                structure     = fun(sample_points) 

                model = NeuralNetwork()
                
                # Define your layers for the model 
                model_struc = nn.ModuleList()

                # first layer
                model_struc.append(nn.Linear(input_sz, structure[0]))

                # hidden layers
                for i, n in enumerate(structure[:-1]):
                    model_struc.append(nn.Linear(structure[i], structure[i+1]))
        
                # output layer
                model_struc.append(nn.Linear(structure[-1], output_sz))
                model_struc.append(nn.Linear(3, output_sz))

                #store the structure into the model 
                model.set_model(model_struc)
                model.set_activation(act)

                #store the model
                models.append(model)
                
    score = -np.inf
    best_model = None

    for model in models:
        optimizer.zero_grad()
        train_model(model,
                    optimizer,
                    criterion,
                    train_data,
                    val_data,
                    n_epochs) 

        evaluation = test_model_performance(model, test_sec)

        if (score < evaluation):
            best_model = model
            score = evaluation
    
    return (score, best_model)

Probamos la función, con dos arquitecturas de redes diferentes y cuatro funciones de activaciones diferentes.

In [ ]:
score, best_model = try_everything(input_sz = 20,
                                   output_sz = 3,
                                   funs = [g1, g2, g3],
                                   lims = [130, 6, 1],
                                   acts = [nn.ReLU(), nn.LeakyReLU(), nn.Sigmoid(), nn.Tanh()],
                                   max_layers = 5,
                                   n_epochs   = 50, 
                                   criterion  = nn.CrossEntropyLoss(), 
                                   data       = (train_sec, val_sec, test_sec))

Como se puede observar el resultado no es que mejore mucho, pero aumenta algo a diferencia de nuestro primer modelo.

In [ ]:
print(f"Nuestro modelo ganador tuvo {score}% de precisión y fue:\n")
print(best_model)

Además, al elegir de manera aleatoria un registro del dataset y utilizar el modelo para predecir su sentimiento, en la mayoría de casos nos está dando la emoción enojado, no entendemos por qué el modelo se cesga a tratar de clasificar los audios como bravos. Nuestra hipótesis es que faltan más datos para que el modelo pueda entender mejor.

Decimos esto en vista de haber probado todas las opciones que se nos ocurrieron; probamos distintas funciones de activación, número distinto de capas, distintas arquitecturas de red neuronal, datos estandarizados y sin estandarizar, sin embargo no fue suficiente.

In [ ]:
a = np.random.randint(test_sec.X.shape[0])
sample = test_sec.X[a,:]

print(sample.reshape((1,-1)))
best_model.make_prediction(sample.reshape((1,-1)))

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=839e86b6-ffa4-40c1-9d7e-74e8c82a69a5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>